In [2]:
import keras
import keras.backend as tf
from keras.datasets import mnist, cifar10
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Input, Dropout, Activation, Conv2D, MaxPooling2D, ZeroPadding2D
import os
import numpy as np
import hickle
from tensorflow import config
os.environ['CUDA_VISIBLE_DEVICES']='1'
# gpus = config.experimental.list_physical_devices(device_type='GPU')
# for gpu in gpus:
#    config.experimental.set_memory_growth(gpu, True)

Using TensorFlow backend.


In [3]:
def build_cifar10model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=(32,32,3),activation='relu'))
  #  model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3),activation='relu'))
   # model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
   # model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3),activation='relu'))
  #  model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    #model.add(Activation('relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
   # model.add(Activation('softmax'))


#     json_string = model.to_json()
#     with open('cifar10.json', 'w') as outfile:
#         json.dump(json_string, outfile)
    return model
#build_cifar10model()

In [4]:
import hickle
from keras.utils.np_utils import to_categorical
def combineandlabeldata(wbs1,zs1,wbs0,zs0):
    layernums = len(wbs1)
    bs = wbs1[0].shape[0] + wbs0[0].shape[0]
    print(bs)
    inputx = [None for _ in range(2*layernums)]
    inputy = [1 for _ in range(wbs1[0].shape[0])] + [0 for _ in range(wbs0[0].shape[0])]
    for i in range(layernums):
        inputx[i] = np.concatenate([wbs1[i],wbs0[i]],axis=0)
        inputx[i+layernums] = np.concatenate([zs1[i],zs0[i]],axis=0)
        

  #  inputy = to_categorical(inputy,2)
    return inputx, inputy


In [5]:
def loadtraindata(indexs):
    xxx = None
    yyy = None
    for i in range(indexs):
        w0 = hickle.load('1www0_{}.hkl'.format(i))
        z0 = hickle.load('1zzz0_{}.hkl'.format(i))
        w1 = hickle.load('1www1_{}.hkl'.format(i))
        z1 = hickle.load('1zzz1_{}.hkl'.format(i))
        xx,yy = combineandlabeldata(w1,z1,w0,z0)
      #  print(yy.shape)
        if xxx is None:
            xxx = xx
            yyy = yy
        else:
            for j in range(len(xxx)):
                
                xxx[j] = np.concatenate([xxx[j],xx[j]],axis=0)
            yyy = np.concatenate([yyy,yy],axis=0)
    return xxx,yyy

def loadtestdata(indexs):
    xxx = None
    yyy = None
    for i in range(indexs-2,indexs):
        w0 = hickle.load('1www0_{}.hkl'.format(i))
        z0 = hickle.load('1zzz0_{}.hkl'.format(i))
        w1 = hickle.load('1www1_{}.hkl'.format(i))
        z1 = hickle.load('1zzz1_{}.hkl'.format(i))
        xx,yy = combineandlabeldata(w1,z1,w0,z0)
      #  print(yy.shape)
        if xxx is None:
            xxx = xx
            yyy = yy
        else:
            for j in range(len(xxx)):
                
                xxx[j] = np.concatenate([xxx[j],xx[j]],axis=0)
            yyy = np.concatenate([yyy,yy],axis=0)
    return xxx,yyy


In [6]:
%%capture
xxx,yyy = loadtraindata(5)
#xxx2,yyy2 = loadtestdata(5)


In [ ]:
bsize= xxx[0].shape[0]


    
shuffleindex = np.random.choice(bsize, bsize, replace=False) 

for index,i in enumerate(xxx):

    xxx[index] = i[shuffleindex]

yyy = yyy[shuffleindex]


In [ ]:
from keras.layers import Conv1D,Reshape,Lambda,Multiply,Add,Concatenate,Permute
from keras.models import Model

def models(inputswb,inputsz):

    
    confidence = []
    inputwbs = []
    inputzs = []
#     inputwbs = Input(shape=(tf.int_shape(inputswb)))
#     inputzs = Input(shape=(tf.int_shape(inputsz)))
    
    for i in range(len(inputswb)):
        
#         inputwb = Lambda(lambda x:x[:,i,:])(inputwbs)
#         inputz = Lambda(lambda x:x[:,i,:])(inputzs)
        inputwb = Input(shape=(tf.int_shape(inputswb[i])))
        inputz = Input(shape=(tf.int_shape(inputsz[i])))
        inputwbs.append(inputwb)
        inputzs.append(inputz)
        
        wb = inputwb
        z = inputz
      #  print(wb.shape,z.shape)
     #   z = tf.squeeze(inputz,axis=1)
     #   print(wb.shape,z.shape)
        t1_1 = Conv1D(16, kernel_size=2, strides=2)(wb)
        t1_1_2 = Conv1D(1, kernel_size=1, strides=1)(t1_1)
        t1_2 = Conv1D(1, kernel_size=1, strides=1)(t1_1_2)
    #    print(t1_2.shape,z.shape)
     #   dwb = Reshape((z.shape[1]+1,10))(t1_2)  #tf.reshape(t1_2,shape=(tf.shape(wb)[0],z.shape[1]+1,10))
        dw = Lambda(lambda x: x[:,:-1])(t1_2)
        db = Lambda(lambda x: x[:,-1])(t1_2)
        #print(dw.shape)
        dw_2 = Reshape(z.shape[1:])(dw)
        #dw = dwb[:,:-1]
        #db = dwb[:,-1]
       # print(z.shape,dw_2.shape)
      #  print(dw.shape,z.shape,db.shape)
        part1 = Multiply()([z,dw_2])
     #   print(part1.shape)
        part1_1 = Lambda(lambda x: tf.sum(tf.batch_flatten(x),axis=1))(part1)
        
       # print(part1_1.shape,db.shape)
        
        part2 = Add()([part1_1,db])
     #   print(part2.shape)
        
        confidence.append(part2)
    confidence_0 = Concatenate(axis=1)(confidence)
  #  confidence_0 = Lambda(lambda x:tf.variable(x))(confidence)
    #print(confidence_0.shape)
   # confidence_1 = Permute((1,0,2))(confidence_0)#Lambda(lambda x:tf.permute_dimensions(x,(1,0,2)))(confidence_0)
    
   # confidence_2 = Reshape(shape=(tf.shape(wb)[0],len(inputswb)*10))(condidence_1)
    #print(confidence_2.shape)
    
    t3 = Dense(16,activation='relu')(confidence_0)
  #  t4 = Dense(8,activation='relu')(t3)
  #  t5 = Dense(8,activation='relu')(t4)
    output = Dense(1,activation='sigmoid')(t3)
    
    return Model(inputwbs+inputzs,output)

In [ ]:
attackmodel = models([i[0] for i in xxx[:len(xxx)//2]],[i[0] for i in xxx[len(xxx)//2:]])

In [ ]:
#bce = keras.losses.BinaryCrossentropy(from_logits=True)


attackmodel.compile(loss=keras.losses.binary_crossentropy,
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])


In [ ]:
%%capture uu
attackmodel.summary()
attackmodel.fit(xxx, yyy,
                  batch_size=64,#steps_per_epoch=200,# 
                  epochs=32,
                  verbose=2,
                 # validation_data=(xxx2,yyy2),
                
               )


In [12]:
uu.stdout

'Model: "model_1"\n__________________________________________________________________________________________________\nLayer (type)                    Output Shape         Param #     Connected to                     \n==================================================================================================\ninput_1 (InputLayer)            (None, 65538, 1)     0                                            \n__________________________________________________________________________________________________\ninput_3 (InputLayer)            (None, 57602, 1)     0                                            \n__________________________________________________________________________________________________\ninput_5 (InputLayer)            (None, 28802, 1)     0                                            \n__________________________________________________________________________________________________\ninput_7 (InputLayer)            (None, 21634, 1)     0                           

import membership_attackload_mnist22
targetmodel = build_cifar10model()

targetmodel.load_weights('shadow_999.h5')

proxymodel = build_cifar10model()
proxymodel.load_weights('proxy_999.h5')


In [19]:

def loadcaldata():
    xxx = [None for _ in range(10)]
  #  yyy = [None for _ in range(10)]
    for i in range(10):
        w = hickle.load('calw_{}.hkl'.format(i))
        z = hickle.load('calz_{}.hkl'.format(i))
        #w1 = hickle.load('1www1_{}.hkl'.format(i))
        #z1 = hickle.load('1zzz1_{}.hkl'.format(i))
      #  xx,yy = combineandlabeldata2(w,z)
      #  print(yy.shape)
#         layernum = w.shape[0]
#         xt = []
#         for j in range(layernum):
#             xt.append(w[j])
#         for j in range(layernum):
#             xt.append(z[j])
            
        xxx[i] = w+z
       # yyy[i] = yy
    return xxx


def getcalvalue():
    values = []
    xxx = loadcaldata()
    for i in range(10):
        calres = attackmodel.predict(xxx[i])
        calres = calres.reshape(-1)
        calres.sort(axis=0)
       # print(calres)
        vv = calres[int(0.90*calres.shape[0])]
        #print(vv)
        values.append(vv)
    return values

values = getcalvalue()
print(values)




[0.77613354, 0.7722682, 0.7892767, 0.7622627, 0.8001671, 0.76460016, 0.8101902, 0.78200316, 0.7836705, 0.7705675]


In [15]:
def loadtestdatabylabel():
    xxx = [None for _ in range(10)]
    yyy = [None for _ in range(10)]
    for i in range(10):
        w0 = hickle.load('testw0_{}.hkl'.format(i))
        z0 = hickle.load('testz0_{}.hkl'.format(i))
        w1 = hickle.load('testw1_{}.hkl'.format(i))
        z1 = hickle.load('testz1_{}.hkl'.format(i))
        xx,yy = combineandlabeldata(w1,z1,w0,z0)
      #  print(yy.shape)
        xxx[i] = xx
        yyy[i] = yy
    return xxx,yyy

def loadtestdataall():
    xxx = None
    yyy = None
    for i in range(10):
        w0 = hickle.load('testw0_{}.hkl'.format(i))
        z0 = hickle.load('testz0_{}.hkl'.format(i))
        w1 = hickle.load('testw1_{}.hkl'.format(i))
        z1 = hickle.load('testz1_{}.hkl'.format(i))
        xx,yy = combineandlabeldata(w1,z1,w0,z0)
      #  print(yy.shape)
        if xxx is None:
            xxx = xx
            yyy = yy
        else:
            for j in range(len(xxx)):
                
                xxx[j] = np.concatenate([xxx[j],xx[j]],axis=0)
            yyy = np.concatenate([yyy,yy],axis=0)
    return xxx,yyy

testxx, testyy = loadtestdataall()

3017
2977
2993
2992
3001
3046
2984
2958
3027
3005


In [17]:



from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
ttt = attackmodel.predict(testxx)

testresa = []

for j in ttt:
    if j >= 0.5:
        testresa.append(1)
    else:
        testresa.append(0)

f1 = f1_score(testyy,testresa)
recall = recall_score(testyy,testresa)
precision = precision_score(testyy,testresa)
accuracy = accuracy_score(testyy,testresa)

print(f1,recall,precision,accuracy)


        

0.8190132962022552 0.9999333333333333 0.6935312341055162 0.7790333333333334


In [20]:

from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

def caltest():
    testxx2,testyy2 = loadtestdatabylabel()
    totaltestres = [] 
    totaltrue = []
    for i in range(10):
        testres = []
        caltest = attackmodel.predict(testxx2[i])
        
        
        for j in caltest:
            if j >= values[i]:
                testres.append(1)
            else:
                testres.append(0)
                
        totaltestres += testres
        totaltrue += testyy2[i]
        
        f1 = f1_score(testyy2[i],testres)
        recall = recall_score(testyy2[i],testres)
        precision = precision_score(testyy2[i],testres)
        accuracy = accuracy_score(testyy2[i],testres)
        
        print(i,f1,recall,precision,accuracy)
        
    f1 = f1_score(totaltrue,totaltestres)
    recall = recall_score(totaltrue,totaltestres)
    precision = precision_score(totaltrue,totaltestres)
    accuracy = accuracy_score(totaltrue,totaltestres)

    print(f1,recall,precision,accuracy)
    
caltest()

3017
2977
2993
2992
3001
3046
2984
2958
3027
3005
0 0.4080865068171132 0.2847769028871391 0.7197346600331676 0.5826980444149817
1 0.3744427934621099 0.25386165211551376 0.7132075471698113 0.5757473967080954
2 0.5738684884713919 0.4438573315719947 0.8115942028985508 0.6665552956899432
3 0.7187993680884676 0.6177868295994569 0.8593012275731823 0.7620320855614974
4 0.48584474885844753 0.3486238532110092 0.8012048192771084 0.6247917360879707
5 0.6914440572977159 0.5886618325642716 0.8377110694183865 0.7383453709783322
6 0.337544214249621 0.2232620320855615 0.6915113871635611 0.560656836461126
7 0.4395180722891566 0.3095723014256619 0.7574750830564784 0.6068289384719405
8 0.330126582278481 0.2180602006688963 0.6791666666666667 0.5629335976214074
9 0.44032334759866854 0.3101138647019424 0.759016393442623 0.6083194675540765
0.49240592930444693 0.35986666666666667 0.7794945848375451 0.6290333333333333


In [ ]:
'''
from attribution.methods import InternalInfluence, AumannShapley



import membership_attackload_mnist22

x_train1, x_test1, y_train1, y_test1 = membership_attackload_mnist22.targetload(database='cifar10')

model1 = build_cifar10model() #model_from_json(json_string)
model1.load_weights('shadow_999.h5')
model1.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.SGD(),
                      metrics=['accuracy'])

from keras import layers, models

input_layer = layers.Input(batch_shape=model1.layers[0].input_shape)
prev_layer = input_layer
for layer in model1.layers:
    prev_layer = layer(prev_layer)

model2 = models.Model([input_layer], [prev_layer])

def _get_inbound_layers(l):

    inbound_layers = []
    for i_node in l._inbound_nodes:
        print(i_node.inbound_layers)
        for i_layer in i_node.inbound_layers:
            print(i_layer)
            inbound_layers.append(i_layer)
    return inbound_layers

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

attr = InternalInfluence(model1, model1.layers[2], Q=None,D='linear_interp',multiply_activation=True).compile().get_attributions(x_train1[:1], batch_size=1)

import keras.backend as K


z = model1.layers[0].output
r = K.variable(1)
baseline = K.variable(K.zeros(shape=K.int_shape(z)[1:]))

b = K.expand_dims(baseline, axis=0)
print('rr')
        # Allocate the alpha term for the interpolation.

a1=1. + K.arange(1.0, dtype='float32')
print('rrr',a1)
a2=K.cast(r, 'float32')
print('rrr2',a2)
a3=K.int_shape(z)[0] 
print('rrr3',a3)
a = K.tile(
     a1/ a2,
    [a3])
print('rrrr')
for _ in range(K.ndim(z) - 1):
    a = K.expand_dims(a, axis=-1)


K.shape(z)
'''